In [1]:
import polars as pl

from src import constants

dataset = pl.scan_csv(constants.OUTPUT_DIR / "dataset.csv").collect()
dataset

session_id,yad_no,session_length,session_interested_sml_cd,session_interested_lrg_cd,last_seen_yad_no,last_seen_sml_cd_0,last_seen_lrg_cd_0,first_seen_yad_no,first_seen_sml_cd_0,first_seen_lrg_cd_0,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd,total_room_cnt_null_flg,wireless_lan_flg_null_flg,kd_stn_5min_null_flg,kd_bch_5min_null_flg,kd_slp_5min_null_flg,kd_conv_walk_5min_null_flg,mean_total_room_cnt_with_onsen_flg,std_total_room_cnt_with_onsen_flg,median_total_room_cnt_with_onsen_flg,mean_total_room_cnt_with_yad_type,std_total_room_cnt_with_yad_type,median_total_room_cnt_with_yad_type,mean_total_room_cnt_with_kd_bch_5min,std_total_room_cnt_with_kd_bch_5min,median_total_room_cnt_with_kd_bch_5min,mean_total_room_cnt_with_kd_stn_5min,std_total_room_cnt_with_kd_stn_5min,median_total_room_cnt_with_kd_stn_5min,mean_total_room_cnt_with_kd_bch_5min_right,std_total_room_cnt_with_kd_bch_5min_right,median_total_room_cnt_with_kd_bch_5min_right,mean_total_room_cnt_with_kd_conv_walk_5min,std_total_room_cnt_with_kd_conv_walk_5min,median_total_room_cnt_with_kd_conv_walk_5min,diff_total_room_cnt_with_onsen_flg
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str


In [2]:
preds = pl.scan_csv(constants.OUTPUT_DIR / "exp000" / "cv_fold0.csv").collect()
preds

session_id,yad_no,predict
str,i64,i64
"""8e42cdceb3d609…",5149,4215
"""8e42cdceb3d609…",5149,5149
"""8e42cdceb3d609…",5149,10554
"""8e42cdceb3d609…",5149,8262
"""8e42cdceb3d609…",5149,10720
"""8e42cdceb3d609…",5149,11671
"""8e42cdceb3d609…",5149,3624
"""8e42cdceb3d609…",5149,13308
"""8e42cdceb3d609…",5149,11211


In [3]:
ds = dataset.group_by("session_id").agg(pl.col("yad_no").alias("candidates"))
ds = ds.join(preds.group_by("session_id").agg(pl.col("yad_no").alias("predictions")), on="session_id")
print(ds)

def intersection(a, b):
    return list(set(a) & set(b))

def has_intersection(a, b):
    return len(intersection(a, b)) > 0

_ds = ds.map_rows(lambda row: has_intersection(row[1], row[2]))
print(_ds.filter(pl.col("map") == True).shape[0] / _ds.shape[0])

shape: (0, 3)
┌────────────┬────────────┬─────────────┐
│ session_id ┆ candidates ┆ predictions │
│ ---        ┆ ---        ┆ ---         │
│ str        ┆ list[str]  ┆ list[i64]   │
╞════════════╪════════════╪═════════════╡
└────────────┴────────────┴─────────────┘


RuntimeError: BindingsError: "Could not determine output type"

In [ ]:
num_session_has_label_in_predict =(
    preds
    .group_by("session_id")
    .map_groups(
        lambda group: group.with_columns(pl.arange(0, group.height).alias("seq_no"))
    )
).filter(pl.col("yad_no") == pl.col("predict"))["session_id"].n_unique()

In [ ]:
num_session_has_label_in_predict / preds["session_id"].n_unique()

0.477753338298204

In [ ]:
preds["session_id"].unique().to_list()[:3]

['8e74132684c7b9a55b2769c454ca082c',
 '3eb36287a40827ccb59e89067c71d30e',
 'f9eff605f0b1b0acdb6c4b98dd83b1b8']

In [ ]:
train_label_df = pl.scan_csv(constants.INPUT_DIR / "train_label.csv").collect()
train_label_df

session_id,yad_no
str,i64
"""000007603d533d…",4101
"""0000ca043ed437…",8253
"""0000d4835cf113…",4863
"""0000fcda1ae1b2…",1652
"""000104bdffaaad…",96
"""00011afe25c343…",4823
"""000125c737df18…",10378
"""0001763050a10b…",10362
"""000178c4d4d567…",1227


In [ ]:
see_targets = [
    "187cbc12d251ad3360f717c1cba7f1f2"
]
for target in see_targets:
    print(target)
    ds_this_target = dataset.filter(pl.col("session_id") == target)
    preds_this_target = preds.filter(pl.col("session_id") == target)
    print("DS: ", ds_this_target)
    print("PREDS: ", preds_this_target)
    yad_candidates = ds_this_target["yad_no"].to_list()
    pred = preds_this_target["predict"].to_list()
    label = train_label_df.filter(pl.col("session_id") == target)["yad_no"].to_list()[0]
    print("YAD CANDIDATES: ", yad_candidates)
    print("PREDS: ", pred)
    print("LABEL: ", label)
    print("Label in candidates: ", label in yad_candidates)
    print("")

187cbc12d251ad3360f717c1cba7f1f2
DS:  shape: (0, 40)
┌────────────┬────────┬────────────┬──────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ session_id ┆ yad_no ┆ session_le ┆ yad_type ┆ … ┆ mean_total ┆ std_total ┆ median_to ┆ diff_tota │
│ ---        ┆ ---    ┆ ngth       ┆ ---      ┆   ┆ _room_cnt_ ┆ _room_cnt ┆ tal_room_ ┆ l_room_cn │
│ str        ┆ i64    ┆ ---        ┆ i64      ┆   ┆ with_kd_co ┆ _with_kd_ ┆ cnt_with_ ┆ t_with_on │
│            ┆        ┆ i64        ┆          ┆   ┆ nv…        ┆ conv_…    ┆ kd_co…    ┆ sen_f…    │
│            ┆        ┆            ┆          ┆   ┆ ---        ┆ ---       ┆ ---       ┆ ---       │
│            ┆        ┆            ┆          ┆   ┆ f64        ┆ f64       ┆ f64       ┆ f64       │
╞════════════╪════════╪════════════╪══════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
└────────────┴────────┴────────────┴──────────┴───┴────────────┴───────────┴───────────┴───────────┘
PREDS:  shape: (0, 3)
┌────────────┬──

In [ ]:
raise Exception()

from tqdm.auto import tqdm

label_in_candidates = 0
unique_session = dataset["session_id"].unique().to_list()
for target in tqdm(unique_session, total=len(unique_session)):
    ds_this_target = dataset.filter(pl.col("session_id") == target)
    preds_this_target = preds.filter(pl.col("session_id") == target)
    yad_candidates = ds_this_target["yad_no"].to_list()
    pred = preds_this_target["predict"].to_list()
    label = train_label_df.filter(pl.col("session_id") == target)["yad_no"].to_list()[0]
    if label in yad_candidates:
        label_in_candidates += 1

print(label_in_candidates / len(dataset["session_id"].unique().to_list()))

Exception: 